In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


In [ ]:
df = pd.read_csv('train.csv')
df.shape

420000 observations, 785 columns

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
# Identify pixel columns
pixel_columns = [f"pixel{i}" for i in range(784)]

# Create a binary matrix: 1 if pixel > 0, else 0
pixel_activity = (df[pixel_columns] > 0).sum(axis=0).values.reshape(28, 28)

# Plot heatmap of active pixel frequencies
plt.figure(figsize=(6, 6))
sns.heatmap(pixel_activity)
plt.title("Pixel Activity Heatmap\n(How Often Each Pixel is Used Across All Digits)")
plt.axis("off")
plt.show()

Variable Creation

In [ ]:
# Identify pixel columns
pixel_columns = [f"pixel{i}" for i in range(784)]

df["total_ink"] = df[pixel_columns].sum(axis=1)
df["avg_intensity"] = df[pixel_columns].mean(axis=1)
df["std_intensity"] = df[pixel_columns].std(axis=1)
df["active_pixels"] = (df[pixel_columns] > 0).sum(axis=1)
df["min_nonzero_intensity"] = df[pixel_columns].replace(0, pd.NA).min(axis=1).fillna(0)
df["max_intensity"] = df[pixel_columns].max(axis=1)

df = df[["total_ink", "avg_intensity", "std_intensity",
         "active_pixels", "min_nonzero_intensity", "max_intensity", "label"]].copy()

df

EDA

In [ ]:
df['label'].value_counts().sort_index().plot(kind='bar')
plt.title('Number of Samples per Digit')
plt.xlabel('Digit')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='label', y='total_ink', data=df)
plt.title('Total Ink Distribution by Digit')
plt.show()


In [ ]:
sns.heatmap(df[["total_ink", "avg_intensity", "std_intensity", "active_pixels", "min_nonzero_intensity", "max_intensity"]].corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlations')
plt.show()

In [ ]:
sns.scatterplot(x="total_ink", y="active_pixels", hue="label", data=df, palette="tab10")
plt.title("Ink Usage vs. Active Pixels by Digit")
plt.show()

In [ ]:
df = pd.read_csv("train.csv")

pixel_columns = [f"pixel{i}" for i in range(784)]
X = df[pixel_columns].values
y = df["label"].values

tsne = TSNE(n_components=2, perplexity=30, n_iter=500, random_state=123)
X_tsne = tsne.fit_transform(X)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y, palette="tab10", s=20, legend='full')
plt.title("t-SNE: 2D Projection of Handwritten Digits")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title="Digit")
plt.tight_layout()
plt.show()


Base Model

In [ ]:
df = pd.read_csv("train.csv")
pixel_columns = [col for col in df.columns if col.startswith("pixel")]

df["total_ink"] = df[pixel_columns].sum(axis=1)
df["avg_intensity"] = df[pixel_columns].mean(axis=1)
df["std_intensity"] = df[pixel_columns].std(axis=1)
df["active_pixels"] = (df[pixel_columns] > 0).sum(axis=1)
df["min_nonzero_intensity"] = df[pixel_columns].replace(0, pd.NA).min(axis=1).fillna(0)
df["max_intensity"] = df[pixel_columns].max(axis=1)

df = df[["total_ink", "avg_intensity", "std_intensity",
         "active_pixels", "min_nonzero_intensity", "max_intensity", "label"]].copy()

feature_cols = ["total_ink", "avg_intensity", "std_intensity",
                 "active_pixels", "min_nonzero_intensity", "max_intensity"]
X = df[feature_cols]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=123),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=123),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    print(f"\n {name} ")
    print(f"Accuracy: {acc:.3f}")
    print(classification_report(y_test, y_pred))
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=range(10)).plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

print("Model Comparison Summary")
for name, acc in results.items():
    print(f"{name}: {acc:.3f}")


Feature Importance

In [ ]:
feature_cols = ["total_ink", "avg_intensity", "std_intensity", "active_pixels", "min_nonzero_intensity", "max_intensity"]
X = df[feature_cols]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Feature importance for Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=123)
rf_model.fit(X_train, y_train)

rf_importance = pd.Series(rf_model.feature_importances_, index=feature_cols).sort_values(ascending=False)
print("Random Forest Feature Importance:")
print(rf_importance)

rf_importance.plot(kind="barh", title="Random Forest - Feature Importance", color="skyblue")
plt.xlabel("Importance")
plt.show()

# Feature Importance for Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=123)
gb_model.fit(X_train, y_train)

gb_importance = pd.Series(gb_model.feature_importances_, index=feature_cols).sort_values(ascending=False)
print("Gradient Boosting Feature Importance:")
print(gb_importance)

gb_importance.plot(kind="barh", title="Gradient Boosting - Feature Importance", color="salmon")
plt.xlabel("Importance")
plt.show()


Modeling

Random Forest (w/o Max Intensity) recommended by feature importance

In [ ]:
# Features without max_intensity
feature_cols_rf = ["total_ink", "avg_intensity", "std_intensity", "active_pixels", "min_nonzero_intensity"]
X_rf = df[feature_cols_rf]

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y, test_size=0.2, random_state=123)

rf_model = RandomForestClassifier(n_estimators=100, random_state=123)
rf_model.fit(X_train_rf, y_train_rf)

y_pred_rf = rf_model.predict(X_test_rf)

acc_rf = accuracy_score(y_test_rf, y_pred_rf)
print("\n Random Forest (without max_intensity)")
print(f"Accuracy: {acc_rf:.3f}")
print(classification_report(y_test_rf, y_pred_rf))


Gradient Boosting Recommended by Feature Importance

In [ ]:
feature_cols_gb = ["std_intensity", "active_pixels"]
X_gb = df[feature_cols_gb]

X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(X_gb, y, test_size=0.2, random_state=123)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=123)
gb_model.fit(X_train_gb, y_train_gb)

y_pred_gb = gb_model.predict(X_test_gb)

acc_gb = accuracy_score(y_test_gb, y_pred_gb)
print("\nGradient Boosting (only std_intensity & active_pixels)")
print(f"Accuracy: {acc_gb:.3f}")
print(classification_report(y_test_gb, y_pred_gb))


Hyperparamter Tuning

Finding the best n_estimator for RandomForest and Gradient Boosting

In [ ]:
errors_rf = []
n_values = range(10, 201, 10)

for n in n_values:
    rf_model = RandomForestClassifier(n_estimators=n, random_state=123)
    rf_model.fit(X_train, y_train)
    errors_rf.append(1 - rf_model.score(X_test, y_test))

errors_gb = []
for n in n_values:
    gb_model = GradientBoostingClassifier(n_estimators=n, random_state=123)
    gb_model.fit(X_train, y_train)
    errors_gb.append(1 - gb_model.score(X_test, y_test))

plt.figure(figsize=(10, 6))
plt.plot(n_values, errors_rf, marker='o', color='skyblue', label="Random Forest")
plt.plot(n_values, errors_gb, marker='o', color='salmon', label="Gradient Boosting")
plt.xlabel("Number of Trees (n_estimators)")
plt.ylabel("Test Error")
plt.title("Test Error vs. n_estimators: Random Forest vs. Gradient Boosting")
plt.legend()
plt.grid(True)
plt.show()

min_error_rf = min(errors_rf)
best_n_rf = n_values[errors_rf.index(min_error_rf)]
print(f"Random Forest: Lowest Test Error = {min_error_rf:.4f} at n_estimators = {best_n_rf}")

min_error_gb = min(errors_gb)
best_n_gb = n_values[errors_gb.index(min_error_gb)]
print(f"Gradient Boosting: Lowest Test Error = {min_error_gb:.4f} at n_estimators = {best_n_gb}")


KNN finding the best K

In [ ]:
# Grid search for best k
param_grid_knn = {'n_neighbors': list(range(1, 21))}  # Try k from 1 to 20

grid_search_knn = GridSearchCV(KNeighborsClassifier(),
                               param_grid_knn,
                               cv=5,
                               scoring='accuracy',
                               n_jobs=-1)

grid_search_knn.fit(X_train, y_train)

# Best k
best_k = grid_search_knn.best_params_['n_neighbors']
best_acc = grid_search_knn.best_score_

print(f"Best k (n_neighbors) for KNN: {best_k}")
print(f"Cross-validated accuracy: {best_acc:.4f}")


Build model using the selected n_estimators and K

Final Model: Random Forest

In [ ]:
# Features without max_intensity
feature_cols_rf = ["total_ink", "avg_intensity", "std_intensity", "active_pixels", "min_nonzero_intensity"]
X_rf = df[feature_cols_rf]

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y, test_size=0.2, random_state=123)

rf_model = RandomForestClassifier(n_estimators=best_n_rf, random_state=123)
rf_model.fit(X_train_rf, y_train_rf)

y_pred_rf = rf_model.predict(X_test_rf)

acc_rf = accuracy_score(y_test_rf, y_pred_rf)
print("\n Final Model: Random Forest (without max_intensity)")
print(f"Accuracy: {acc_rf:.3f}")
print(classification_report(y_test_rf, y_pred_rf))

Final Model: Gradient Boosting

In [ ]:
feature_cols_gb = ["std_intensity", "active_pixels"]
X_gb = df[feature_cols_gb]

X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(X_gb, y, test_size=0.2, random_state=123)

gb_model = GradientBoostingClassifier(n_estimators=best_n_gb, random_state=123)
gb_model.fit(X_train_gb, y_train_gb)

y_pred_gb = gb_model.predict(X_test_gb)

acc_gb = accuracy_score(y_test_gb, y_pred_gb)
print("\nFinal Model: Gradient Boosting (only std_intensity & active_pixels)")
print(f"Accuracy: {acc_gb:.3f}")
print(classification_report(y_test_gb, y_pred_gb))

In [ ]:
feature_cols = ["total_ink", "avg_intensity", "std_intensity", "active_pixels", "min_nonzero_intensity", "max_intensity"]
X = df[feature_cols]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

models = {
    "KNN": KNeighborsClassifier(n_neighbors=best_k)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.3f}")
    print(classification_report(y_test, y_pred))
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=range(10)).plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

print("\nModel Comparison Summary")
for name, acc in results.items():
    print(f"{name}: {acc:.3f}")